<a href="https://colab.research.google.com/github/Whoami666/HSE/blob/main/pubmed_project/Parse_pubmed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Для качественного анализа нужен сбор более 100тыс. статей, поэтому необходимо найти базу данных, к которой можно обращаться

**Google Scholar** - самая крупная база публикаций, содержит много "мусора", а также не имеет бесплатных библиотек, которые позволяют совершать много запросов (более 1000 в день). Платные решения для работы с данными в Google Scholar стоят слишком дорого, чтобы приобретаться для студенческой работы

**Pubmed** - самая крупная база медицинских исследований
Преимущества:


*   Нет ограничений по количеству запросов, идет с бесплатной библиотекой для парсинга => есть возможность загрузить сотни тыс. статей
*   Содержит валидные ссылки на статьи, хорошие абстракты



**План работы**

*   Взять статьи с 2006 года до 2023 года включительно
*   Загрузить статьи в 1 базу данных (программирование)
*   Провести сетевой анализ
* Найти финансовые да



**Планируемые результаты**


*   Выделить основные тематики исследований в области болезни Альцгеймера
*   Идентифицировать статьи, которые связаны со скандалом с подделкой данных 2006 года, оценить их эффект на область
*   Найти крупные гранты, выделенные на разные области исследований на эту тему, оценить насколько они корелируют с найденными индексами
*  Оценить статьи, которые имеют наибольший эффект на область => эти статьи нужно проверять более пристально





В работе я применяю базу данных **Entrez**, которая помогает работать с Pubmed

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install bio

In [ ]:
import requests
from Bio import Entrez
import pandas as pd
import time
import gdown
from http.client import IncompleteRead

In [ ]:
def search_pubmed(query, start_year, end_year, start_month, end_month ): #, start_day, end_day
    Entrez.email = "your-email@example.com"  # Replace with your email address

    # Build the search query with year, month, and day filters
    search_query = f'{query} AND ("{start_year}/{start_month}"[Date - Publication] : "{end_year}/{end_month}"[Date - Publication])'
    # Perform the search
    handle = Entrez.esearch(db="pubmed", term=search_query, retmax=1000000)  # Limiting to 10 results for this example
    record = Entrez.read(handle)
    handle.close()

    return record["IdList"]

if __name__ == "__main__":
    query = "Alzheimer's disease"
    start_year = "2008"
    end_year = "2008"
    start_month = "01"  # January
    end_month = "12"   # December
  # start_day = "01"    # 1st day of the month
  #  end_day = "30"

    pmids = search_pubmed(query, start_year, end_year, start_month, end_month) #, start_day, end_day

    print(f"PubMed IDs for papers on {query} published between {start_year}/{start_month} and {end_year}/{end_month}:")
    print(len(pmids))
    print(len(set(pmids)))


In [ ]:
df = pd.DataFrame()

In [ ]:
handle = Entrez.esummary(db="pubmed", id=pmids, retmode="xml")
records = Entrez.parse(handle)

In [ ]:
my_list = list(records)

In [ ]:
len(my_list)

In [ ]:
try:
  i = 0
  for record in my_list:
  # each record is a Python dictionary or list.
    try:
        #достаем цитирования
        i += 1
        handle_link = Entrez.elink(dbfrom="pubmed", id=record['Id'], linkname="pubmed_pubmed_refs")
        record_link = Entrez.read(handle_link)
        handle_link.close()
        all_pmids = [link['Id'] for link_dict in record_link[0]['LinkSetDb'] for link in link_dict['Link']]

        #достаем абстракты
        pmids_id = [record['Id']]
        handle_fetch = Entrez.efetch(db="pubmed", id=','.join(map(str, pmids_id)),
                              rettype="xml", retmode="text")
        records_fetch = Entrez.read(handle_fetch)
        handle_fetch.close()

        try:
          abstracts = [pubmed_article['MedlineCitation']['Article']['Abstract']['AbstractText'][0]
                      for pubmed_article in records_fetch['PubmedArticle']]
          if len(abstracts) > 0:
            abstract = abstracts[0]
          else:
            abstract = ""
        except KeyError as e:
            abstract = ""

        pub_info = {
            'Id': record['Id'],
            'PubDate': record['PubDate'],
            'Source': 'JAMA Neurol',
            'AuthorList': record['AuthorList'],
            'LastAuthor': record['LastAuthor'],
            'Title': record['Title'],
            'Volume': record['Volume'],
            'Issue': record['Issue'],
            'Pages': record['Pages'],
            'LangList': record['LangList'],
            'NlmUniqueID': record['NlmUniqueID'],
            'ISSN': record['ISSN'],
            'ESSN': record['ESSN'],
            'PubTypeList': record['PubTypeList'],
            'RecordStatus': record['RecordStatus'],
            'PubStatus': record['PubStatus'],
            'DOI':record['ELocationID'],
            'References': record['References'],
            'HasAbstract': record['HasAbstract'],
            'PmcRefCount': record['PmcRefCount'],
            'FullJournalName': record['FullJournalName'],
            'Citations': all_pmids,
            'Abstract': abstract
        }

        new_df = pd.DataFrame([pub_info])
        df = pd.concat([df, new_df], ignore_index=True)

        if i % 10 == 0:
          time.sleep(1)
        else:
          time.sleep(0.2)

    except Exception as e:
      print(f"Error: {e}. Skipping {record['Id']}")
      time.sleep(0.5)
      continue

except IncompleteRead:
  print('IncompleteRead')
  #   time.sleep(0.05)



In [ ]:
df.head(1)

In [ ]:
df.info()

In [ ]:
df.to_excel('pubmed2008.xlsx', index=False)

In [ ]:
!cp pubmed2008.xlsx /content/drive/MyDrive

In [ ]:
a = ['17516724', '15326236', '15260515', '15220031', '15156547', '14992980', '14734594', '14704391', '14557565', '14533127', '14512822', '12925369', '12917900', '12891643', '12875613', '12860500', '12757558', '12745762', '12672860', '12670060', '12627986', '12580704', '12473759', '12457078', '12433264', '12145455', '12141890', '12076218', '12028215', '11994927', '11988038', '11925682', '11722692', '11571319', '11571318', '11524468', '11502918', '11445614', '11343622', '11342679', '11255445', '11142178', '11110737', '11096224', '11081959', '10971049', '10947490', '10885864', '10881251', '10881250', '10855599', '10823365', '10645761', '10600044', '10484247', '10389036', '10323651', '10219924', '10214734', '10211151', '10172777', '10172695', '10066203', '9916370', '9757169', '9737841', '9736873', '9736007', '9695032', '9674763', '9543467', '9437448', '9393382', '9344401', '9343469', '9343467', '9283930', '9193203', '9129864', '9127979', '9110909', '9052710', '8940320', '8670538', '8375811', '8131123', '8059882', '8005496', '8005492', '7856990', '7774731', '7715060', '6610841', '3124195', '2960595', '2776169', '2771064', '2299115', '2233632']

In [ ]:
len(a)

In [ ]:
a = ['36574591', '36750875', '34661615', '34494715', '35130933', '35078917', '32988409', '35491781', '37268240', '33427873', '33210117', '36547971', '34928318', '36074638', '33431793', '35084051', '36880887', '35950735', '36447478', '34187867', '32333900', '37079306', '37667408', '37924152', '33491853', '34542072', '36450604', '35450967', '33766131', '36876954', '35476397', '37787146', '36745413', '36229847', '33106600', '33419453', '35383826', '36776057', '35603139', '36789483', '33436035', '37269398', '37874105', '33773595', '36371232', '35847667', '37645764', '31009028', '37065463', '36229187', '36221099', '36408096', '35234335', '35789365', '37016464', '36002891', '37742656', '37489795', '32722745', '33792168', '26401938', '35022305', '36937522', '34225797', '31350262', '36648169', '35098969', '36062146', '36626935', '37009461', '34120152', '34151519', '35401142', '35288777', '35872565', '35226303', '36544221', '37087687', '31233127', '37666862', '36878697', '37399739', '34309632', '37202168', '37294682', '35502634', '37652185', '37491358', '34180956', '28869470', '33585983', '34418401', '37690071', '28346578', '35195758', '36604729', '34654465', '36120776', '32450185', '32391858', '33257949', '34593971', '37671801', '35577574', '37578787', '32986672', '37899059', '37580163', '35550625', '35641311', '37216632', '37475029', '31787105', '36261295', '35023610', '36709581', '37500404', '36988152', '37784138', '37885919', '37038190', '36711545', '37723371', '37639492', '37875471', '36253968', '36192174', '34906229', '37038810', '36597122', '37797300', '36744337', '36576155', '37530130', '31371569', '34310832', '37637274', '36946452', '37716237', '34389066', '37852045', '37506291', '37381042', '37502842', '37762457', '36152307', '33443136', '36847009', '37279785', '37740209', '37519166', '37932720', '37400888', '37852722', '32591471', '37264656', '37534889', '35103843', '36441007', '36617786', '37469165', '37801487', '33663013', '36476732', '37762278', '36575455', '37279390', '37662203', '37202169', '35510092', '37748892', '34786815', '37919158', '36517236', '37718807', '33789730', '37932961', '33199433', '37254223', '34302785', '33090679', '37647995', '37951931', '37939982', '37480401', '36919582', '37021572', '36710682', '35880989', '37950758', '29071500', '37638636', '35554506', '37180996', '37274930', '37208024', '37574735', '37078495', '37833041', '36882326', '37749258', '35659284', '35278341', '31182505', '37656487', '34002480', '37653254', '37283062', '35952440', '36174450', '37723414', '35155728', '37031430', '37208383', '34934798', '37880775', '37131622', '37491668', '35260470', '37821235', '35820077', '31985466', '36641673', '37759100', '33746700', '37482997', '34563258', '37813589', '37431060', '36640138', '37697307', '37762384', '37144597', '34627371', '37646421', '37818793', '37915896', '36178120', '37688927', '37424464', '37828917', '37876071', '37690109', '37936180', '37884086', '34517889', '37828595', '37499883', '31958084', '37622539', '34311775', '35877814', '30124772', '34966024', '34899299', '35073974', '35572134', '37610594', '34921102', '29889074', '37118088', '31853477', '37584311', '37801885', '37481959', '34713772', '37066906', '35222254', '37263261', '37898567', '35369719', '37608748', '33952156', '37079778', '35833836', '37804609', '35137414', '35356487', '37873312', '36041868', '35346695', '36852497', '37925455', '37925995', '37392405', '37641428', '33674752', '35858992', '22163278', '37946268', '37889270', '37897306', '37334604', '37369876', '37858957', '37752907', '37788938', '36610788', '36737481', '37870702', '37555137', '35089640', '37817991', '22801742', '32925032', '37679610', '32699143', '37882772', '37768470', '37541133', '37778613', '37759734', '36506473', '37933726', '37920945', '30859121', '31920598', '32692110', '31700988', '36820052', '37595816', '37759493', '37603931', '37876876', '37502947', '37394144', '37140203', '37550958', '37459658', '37694362', '35197838', '37645867', '31011623', '31456676', '37838715', '37685872', '30664784', '36056631', '37610059', '37924925', '36316116', '33813555', '37746151', '33797846', '36092690', '37742223', '37698523', '37901792', '37341347', '37703428', '37067597', '35800899', '35862098', '35418452', '37220890', '32582835', '37581299', '36295851', '37849637', '37232524', '37843431', '36081458', '37196122', '37519456', '31673597', '37534390', '36867286', '37842184', '34661109', '37543169', '35723107', '37879623', '34122044', '37801139', '37162971', '31879236', '36950699', '37257406', '37805158', '36993907', '37762074', '37872582', '37364279', '37675435', '37683329', '37166032', '35360215', '36041549', '37090601', '37864751', '37700370', '33692682', '28793924', '37939874', '35867329', '37607528', '35313387', '37950256', '37917365', '34192666', '36110031', '32584071', '37710290', '37762467', '30175226', '34776937', '37658429', '34338426', '37652704', '37694991', '37398438', '37827850', '37521522', '26543800', '37932910', '35369094', '35617251', '37099313', '27722193', '37679434', '37333223', '36514476', '30386818', '33748396', '32607407', '36065116', '37702279', '37195150', '37948492', '37896129', '36132375', '37766242', '37645993', '35496372', '37718801', '34945719', '37901041', '37775501', '37244142']

In [ ]:
for i in a:
  print(i
        )

In [ ]:
len(a)

In [ ]:
if '28822751' in a:
  print("pls")

In [ ]:


# Shareable link to the folder in your Google Drive where you want to upload the file
drive_folder_link = "https://drive.google.com/drive/folders/1MZmLNutZTkx6AsJiOLHdneae93kmCH2r?usp=sharing"

# URL to download the file (use the file ID from the shareable link)
file_url = "https://drive.google.com/uc?id=2023Jan.csv"

# Download the file to Colab
gdown.download(file_url, output='my_data.csv', quiet=False)

# Move the file to the desired folder



In [ ]:
!mv my_data.csv /content/drive/My\ Drive/YOUR_FOLDER_NAME/my_data.csv